In [1]:
# Verify we're in the correct working directory
import os
os.getcwd()   

'/Users/seraphinashi/Desktop/l3c_ctml'

In [2]:
# pip install -r requirements.txt

In [3]:
from src.STEP1_feature import a_cohort as a
from src.STEP1_feature import b_medication as b
from src.STEP1_feature import c_diagnosis as c
from src.STEP1_feature import d_lab_measures as d
from src.STEP1_feature import e_comorbidity as e
from src.STEP1_feature import i_combine_synthetic_data as i
import os
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

In [4]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("l3c_ctml") \
        .getOrCreate()

# Read CSV file into table
data_path = "synthetic_data/training"

schema = StructType([
    StructField('table', StringType(), True),
    StructField('field', StringType(), True),
    StructField('type', StringType(), True)
])

all_fields = spark.createDataFrame([], schema)
    
for path, subdirs, files in os.walk(data_path):
    for name in files:
        if "csv" in name and name[0]!=".":
            # table_fields = [(table_name, f.name, str(f.dataType)) for f in datafile.schema.fields]
            full_file = (os.path.join(path, name))
            df = spark.read.csv(os.path.join(path, name), header=True, inferSchema=True)
            var_name = name[:-4]
            print(full_file)
            globals()[var_name] = df

synthetic_data/training/observation_period.csv
synthetic_data/training/drug_exposure.csv
synthetic_data/training/death.csv
synthetic_data/training/concept.csv
synthetic_data/training/measurement.csv
synthetic_data/training/condition_occurrence.csv
synthetic_data/training/microvisits_to_macrovisits.csv
synthetic_data/training/visit_occurrence.csv
synthetic_data/training/long_covid_silver_standard.csv
synthetic_data/training/person.csv
synthetic_data/training/observation.csv
synthetic_data/training/procedure_occurrence.csv


In [5]:
# 1_cohort_and_features

covid_pasc_index_dates = a.sql_statement_01(long_covid_silver_standard)
cohort = a.sql_statement_00(covid_pasc_index_dates, person)
long_covid_patients = a.sql_statement_08(covid_pasc_index_dates)

hosp_cases = a.sql_statement_04(cohort, condition_occurrence, microvisits_to_macrovisits)

hosp_and_non = a.sql_statement_06(cohort, hosp_cases)

Feature_Table_Builder_v0 = a.sql_statement_03(covid_pasc_index_dates, hosp_and_non, microvisits_to_macrovisits)
icu_visits = a.sql_statement_05(microvisits_to_macrovisits, concept) # empty for some reason
inpatient_visits = a.sql_statement_07(microvisits_to_macrovisits, concept)

tot_icu_days_calc = a.sql_statement_09(Feature_Table_Builder_v0, icu_visits)
tot_ip_days_calc = a.sql_statement_10(Feature_Table_Builder_v0, inpatient_visits)

Feature_Table_Builder = a.sql_statement_02(Feature_Table_Builder_v0, tot_icu_days_calc, tot_ip_days_calc)

In [6]:
# 2_med_feature_table
DrugConcepts = b.sql_statement_00(concept)
Drugs_for_These_Patients = b.sql_statement_01(Feature_Table_Builder, drug_exposure)

drugRollUp = b.sql_statement_04(DrugConcepts, Drugs_for_These_Patients)

covid_drugs = b.sql_statement_02(Feature_Table_Builder, drugRollUp)
pre_pre_drugs = b.sql_statement_10(Feature_Table_Builder, drugRollUp)
pre_drugs = b.sql_statement_07(Feature_Table_Builder, drugRollUp)
post_drugs = b.sql_statement_05(Feature_Table_Builder, drugRollUp)

covidtbl = b.sql_statement_03(Feature_Table_Builder, covid_drugs, microvisits_to_macrovisits)
prepretbl = b.sql_statement_11(Feature_Table_Builder, covid_drugs, microvisits_to_macrovisits)
pretbl = b.sql_statement_12(Feature_Table_Builder, pre_pre_drugs, microvisits_to_macrovisits)
posttbl = b.sql_statement_06(Feature_Table_Builder, post_drugs, microvisits_to_macrovisits)

pre_post_med_count = b.sql_statement_08(covidtbl, posttbl, prepretbl, pretbl)

pre_post_med_count_clean = b.sql_statement_09(Feature_Table_Builder, pre_post_med_count)

In [7]:
pre_post_med_count_clean

DataFrame[person_id: double, patient_group: string, ancestor_drug_concept_name: string, ancestor_drug_concept_id: int, pre_pre_med_count: bigint, pre_med_count: bigint, covid_med_count: bigint, post_med_count: bigint, apprx_age: int, sex: int, race: int, ethn: int, tot_long_data_days: bigint, op_post_visit_ratio: double, post_ip_visit_ratio: double, covid_ip_visit_ratio: double, post_icu_visit_ratio: double, covid_icu_visit_ratio: double]

In [8]:
# 3_dx_feature_table
pre_pre_condition = c.sql_statement_05(Feature_Table_Builder, condition_occurrence)
pre_condition = c.sql_statement_03(Feature_Table_Builder, condition_occurrence)
covid_condition = c.sql_statement_00(Feature_Table_Builder, condition_occurrence)
post_condition = c.sql_statement_02(Feature_Table_Builder, condition_occurrence)

four_windows_dx_counts = c.sql_statement_01(Feature_Table_Builder, microvisits_to_macrovisits, pre_pre_condition, pre_condition, covid_condition, post_condition)

pre_post_dx_count_clean = c.sql_statement_04(Feature_Table_Builder, four_windows_dx_counts)


In [9]:
# 4_lab_measure_table
measure_person = d.measurement_person(measurement, Feature_Table_Builder, concept)
pre_pre_measurement = d.sql_statement_05(Feature_Table_Builder, measure_person )

pre_measurement = d.sql_statement_04( Feature_Table_Builder, measure_person )
covid_measurement = d.sql_statement_00(Feature_Table_Builder, measure_person )
post_measurement = d.sql_statement_03(Feature_Table_Builder, measure_person )

four_windows_measure = d.sql_statement_01(covid_measurement, post_measurement, pre_measurement, pre_pre_measurement)

lab_measures_clean = d.sql_statement_02(four_windows_measure)

# 5_comorbidity_table
high_level_condition_occur = e.sql_statement_01(Feature_Table_Builder, condition_occurrence, concept)

comorbidity_counts = e.sql_statement_00(Feature_Table_Builder, high_level_condition_occur)

In [10]:
# 6_covid_measures

In [11]:
# 9_obs

In [12]:
# i_combine

In [13]:
pre_post_med_final_distinct = i.pre_post_med_final_distinct(pre_post_med_count_clean)
add_labels = i.add_labels(pre_post_dx_count_clean, pre_post_med_count_clean, long_covid_patients, Feature_Table_Builder)
#--------------------------

NameError: name 'spark' is not defined

In [ ]:
condition_rollup = i.condition_rollup(long_covid_patients, pre_post_dx_count_clean, concept)

parent_condition_rollup = i.parent_condition_rollup(condition_rollup)

final_rollups = i.final_rollups(condition_rollup, parent_condition_rollup)
pre_post_more_in_dx_calc = i.pre_post_more_in_dx_calc(pre_post_dx_count_clean)

add_alt_rollup = i.add_alt_rollup(final_rollups, pre_post_more_in_dx_calc)

pre_post_dx_final = i.pre_post_dx_final(add_alt_rollup)
#--------------------------

In [ ]:
import pandas as pd
import re

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.context import SparkContext
ctx = SparkContext.getOrCreate()